<a href="https://colab.research.google.com/github/Cristopher-Quirarte/CristopherQG.github.io/blob/master/Diabetes_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbolic Transformer


In [50]:
import numpy as np
import pandas as pd

from sklearn.utils import check_random_state
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

In [68]:
!pip install gplearn

In [69]:
from gplearn.genetic import SymbolicTransformer

In [8]:
rng = check_random_state(0)

In [40]:
diabetes = load_diabetes( return_X_y=False, as_frame=True, scaled=True)

In [24]:
type(diabetes)
#pd.DataFrame(diabetes)

sklearn.utils._bunch.Bunch

In [14]:
diabetes.data = diabetes.data[perm]
diabetes.target = diabetes.target[perm]

In [46]:
diabetes_pd= pd.DataFrame(data = np.c_[diabetes['data'],diabetes['target']], columns = diabetes['feature_names'] + ['target'])
diabetes_pd.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [47]:
diabetes_pd.shape

(442, 11)

In [48]:
diabetes_pd.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,442.000000
mean,-2.511817e-19,1.230790e-17,-2.245564e-16,-4.797570e-17,-1.381499e-17,3.918434e-17,-5.777179e-18,-9.042540e-18,9.293722e-17,1.130318e-17,152.133484
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,77.093005
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123988e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260971e-01,-1.377672e-01,25.000000
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665608e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324559e-02,-3.317903e-02,87.000000
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670422e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947171e-03,-1.077698e-03,140.500000
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564379e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243232e-02,2.791705e-02,211.500000
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320436e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335973e-01,1.356118e-01,346.000000


Using Ridge Regression for the first 300 of the samples and see how it performs with the rest of 142

In [51]:
est = Ridge()

In [60]:
diabetes_2 = load_diabetes()
# importing again because the original diabetes was used for anaylisis and exploration
# The diabetes original set can't be used by Ridge method

In [75]:
est.fit(diabetes_2.data[:300, :], diabetes_2.target[:300])

Ridge()

In [76]:
print(est.score(diabetes_2.data[300:, :], diabetes_2.target[300:]))

0.4369227721575072


So now we’ll train our transformer on the same first 300 samples to generate some new features. Let’s use a large population of 2000 individuals over 20 generations. We’ll select the best 100 of these for the hall_of_fame, and then use the least-correlated 10 as our new features. A little parsimony should control bloat, but we’ll leave the rest of the evolution options at their defaults. The default metric='pearson' is appropriate here since we are using a linear model as the estimator. If we were going to use a tree-based estimator, the Spearman correlation might be interesting to try out too:

In [77]:
function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min']

In [78]:
gp = SymbolicTransformer(generations=20, population_size=2000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, n_jobs=3)

In [79]:
gp.fit(diabetes_2.data[:300, :], diabetes_2.target[:300])

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.37         0.123529        5          0.66116         0.790509      1.61m
   1     6.64         0.354131        5         0.699897         0.447698     44.28s
   2     5.38         0.501288        5         0.713058          0.32527     36.78s
   3     4.42         0.639623        3         0.711019         0.335277     34.53s
   4     4.28         0.654499        3         0.720537         0.240732     34.85s
   5     4.44         0.654701        3         0.714426         0.174398     36.59s
   6     4.53         0.657543       15         0.713256         0.360926     41.53s
   7     4.68         0.657978        6         0.714965          0.41055     38.52s
   8     5.14         0.663026        7          0.71748         0.374692  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                  'abs', 'neg', 'inv', 'max', 'min'],
                    max_samples=0.9, n_jobs=3, parsimony_coefficient=0.0005,
                    population_size=2000, random_state=0, verbose=1)

In [80]:
gp_features = gp.transform(diabetes_2.data)
new_diabetes = np.hstack((diabetes_2.data, gp_features))

In [83]:
est = Ridge()
est.fit(new_diabetes[:300, :], diabetes.target[:300])
print(est.score(new_diabetes[300:, :], diabetes.target[300:]))

0.47539049994594973
